In [1]:
import numpy as np
import matplotlib.pyplot as plt
from imp import reload

%matplotlib inline

In [2]:
### Read-in experiment config
from collections import namedtuple
from tda.experiments.ocsvm_detector.main_mnist_lenet import experiment_plan
from tda.embeddings import ThresholdStrategy

config = experiment_plan.experiments[0].config

# fix some quirks
config["n_jobs"] = 4  # increase, if running on a big machine
config["all_eps"] = [.01, .02, .05, .1, .2, .3, .4]

if "mnist_lenet" in config["architecture"]:
    config["epochs"] = 50
    config["threshold_strategy"] = ThresholdStrategy.ActivationValue
    config["thresholds"] = "_".join(["0.1"] * 4)
elif "svhn" in config["architecture"]:
    config["epochs"] = 300
else:
    raise NotImplementedError(config["architecture"])

# convert config to namedtuple
Config = namedtuple("Config", sorted(config))
config = Config(**config)

2020-05-22 20:02:30,607 - Devices - INFO - Found 0 devices compatible with CUDA
2020-05-22 20:02:31,890 - Cache - INFO - Cache root /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache/


[{'embedding_type': 'PersistentDiagram'}]


In [3]:
# Build dataset and model architecture
from tda.models.architectures import get_architecture
from tda.models import get_deep_model, Dataset

architecture = get_architecture(config.architecture)
dataset = Dataset.get_or_create(name=config.dataset)

architecture = get_deep_model(
    num_epochs=config.epochs,
    dataset=dataset,
    architecture=architecture)

2020-05-22 20:02:40,198 - Datasets - INFO - Instantiated dataset MNIST with validation_size 1000
/home/elvis/anaconda3/envs/art/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'tda.models.architectures.architecture.Architecture' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/elvis/anaconda3/envs/art/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/elvis/anaconda3/envs/art/lib/python3.7/site-packages/torch/serialization.py:493: SourceChange

In [4]:
# Check that everything looks okay
from tda.graph import Graph

x = dataset.train_dataset[0][0]
graph = Graph.from_architecture_and_data_point(architecture, x)
for key in graph._edge_dict:
    layer_matrix = graph._edge_dict[key]
    print(layer_matrix.shape)

(5760, 784)
(1440, 5760)
(1280, 1440)
(320, 1280)
(50, 320)
(10, 50)


In [5]:
# Compute thresholds
from tda.threshold_underoptimized_edges import process_thresholds_underopt
from tda.thresholds import process_thresholds

thresholds = None
edges_to_keep = None
raw_thresholds = config.thresholds
if config.threshold_strategy == ThresholdStrategy.ActivationValue:
    if ":" in raw_thresholds:
        raw_thresholds = "_".join(raw_thresholds.split(":"))
    thresholds = process_thresholds(architecture=architecture,
                                    dataset=dataset,
                                    raw_thresholds=raw_thresholds,
                                    dataset_size=10  # for reduced mem
                                    )
elif config.threshold_strategy == ThresholdStrategy.UnderoptimizedMagnitudeIncrease:
    edges_to_keep = process_thresholds_underopt(
        architecture=architecture,
        raw_thresholds=raw_thresholds,
        method=config.threshold_strategy,
        thresholds_are_low_pass=True)
else:
    raise NotImplementedError(config.threshold_strategy)
if thresholds is not None:
    print("Edge thresholds for activation graphs:")
    for (src, dst), val in thresholds.items():
        print("  %s --> %s: %.2f" % (src + 1, dst + 1, val))

2020-05-22 20:02:41,040 - Thresholds - INFO - Detected legacy format for thresholds
2020-05-22 20:02:41,041 - Thresholds - INFO - My received thresholds {(-1, 0): 0.1, (0, 1): 0.1, (1, 2): 0.1, (2, 3): 0.1}
2020-05-22 20:02:41,048 - GraphDataset - INFO - Using source dataset mnist
2020-05-22 20:02:41,049 - GraphDataset - INFO - Checking that the received architecture has been trained
2020-05-22 20:02:41,049 - GraphDataset - INFO - OK ! Architecture is ready
2020-05-22 20:02:41,049 - GraphDataset - INFO - I am going to generate a dataset of 10 points...
2020-05-22 20:02:41,050 - GraphDataset - INFO - Only successful adversaries ? no
2020-05-22 20:02:41,050 - GraphDataset - INFO - Which attack ? FGSM
2020-05-22 20:02:41,475 - GraphDataset - INFO - computing sample number = 10/10
2020-05-22 20:02:41,531 - Cache - INFO - Creating cache file /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache//get_sample_dataset/adv=False_archi=mnist_lenet_e_50_compute_graph=True_dataset=mnist_datas

Edge thresholds for activation graphs:
  0 --> 1: 17354.22
  1 --> 2: 63804.91
  2 --> 3: 4958.86
  3 --> 4: 591506.34


In [6]:
# Build tda dataset (i.e activation graphs for clean and adversarial inputs)
from tda.protocol import get_protocolar_datasets

# %debug
import torch
X_train, _ = zip(*dataset.train_dataset)
X_train = torch.stack(X_train)
lims = X_train.min().item(), X_train.max().item()
all_epsilons = [0.01, .02, 0.05, 0.1, 0.2, 0.3, 0.4]
(train_clean, test_clean, train_adv,
 test_adv) = get_protocolar_datasets(dataset=dataset,
                                     succ_adv=True,
                                     dataset_size=config.dataset_size,
                                     noise=0., compute_graph=False,
                                     all_epsilons=all_epsilons,
                                     attack_type="FGSM",
                                     archi=architecture,
                                     lims=lims)

2020-05-22 20:03:02,616 - C3PO - INFO - I will produce for you the protocolar datasets !
2020-05-22 20:03:02,616 - GraphDataset - INFO - Using source dataset mnist
2020-05-22 20:03:02,617 - GraphDataset - INFO - Checking that the received architecture has been trained
2020-05-22 20:03:02,617 - GraphDataset - INFO - OK ! Architecture is ready
2020-05-22 20:03:02,617 - GraphDataset - INFO - I am going to generate a dataset of 50 points...
2020-05-22 20:03:02,618 - GraphDataset - INFO - Only successful adversaries ? yes
2020-05-22 20:03:02,618 - GraphDataset - INFO - Which attack ? FGSM
2020-05-22 20:03:02,626 - GraphDataset - INFO - computing sample number = 10/50
2020-05-22 20:03:02,633 - GraphDataset - INFO - computing sample number = 20/50
2020-05-22 20:03:02,640 - GraphDataset - INFO - computing sample number = 30/50
2020-05-22 20:03:02,646 - GraphDataset - INFO - computing sample number = 40/50
2020-05-22 20:03:02,653 - GraphDataset - INFO - computing sample number = 50/50
2020-05-2

2020-05-22 20:03:45,219 - GraphDataset - INFO - Which attack ? FGSM
2020-05-22 20:03:45,316 - GraphDataset - INFO - computing sample number = 10/100
2020-05-22 20:03:45,492 - GraphDataset - INFO - computing sample number = 20/100
2020-05-22 20:03:45,666 - GraphDataset - INFO - computing sample number = 30/100
2020-05-22 20:03:45,755 - GraphDataset - INFO - computing sample number = 40/100
2020-05-22 20:03:45,829 - GraphDataset - INFO - computing sample number = 50/100
2020-05-22 20:03:45,937 - GraphDataset - INFO - computing sample number = 60/100
2020-05-22 20:03:46,050 - GraphDataset - INFO - computing sample number = 70/100
2020-05-22 20:03:46,259 - GraphDataset - INFO - computing sample number = 80/100
2020-05-22 20:03:46,419 - GraphDataset - INFO - computing sample number = 90/100
2020-05-22 20:03:46,526 - GraphDataset - INFO - computing sample number = 100/100
2020-05-22 20:03:46,527 - Cache - INFO - Creating cache file /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../cache/

In [7]:
# Compute embeddings for test-set adversarial inputs
from tda.embeddings import (get_embedding, EmbeddingType,
                            KernelType, ThresholdStrategy)
from joblib import delayed, Parallel


def embedding_getter(line):
    embedding = get_embedding(
        architecture=architecture,
        embedding_type=EmbeddingType.PersistentDiagram,
        line=line, dataset=dataset,
        threshold_strategy=config.threshold_strategy,
        thresholds=thresholds, edges_to_keep=edges_to_keep)
    print(".", end="")
    return embedding

embeddings = {}
for eps in test_adv:
    print("\nComputing test adversarial embeddings for eps=%.3f" % eps)
    embeddings[eps] = Parallel(n_jobs=config.n_jobs)(
        delayed(embedding_getter)(line)
        for line in test_adv[eps])


Computing test adversarial embeddings for eps=0.010


KeyboardInterrupt: 

In [ ]:
embeddings[0] = Parallel(n_jobs=config.n_jobs)(
        delayed(embedding_getter)(line)
        for line in test_clean)

In [ ]:
embeddings_test_adv = dict((eps, embeddings[eps]) for eps in embeddings if eps != 0)

In [ ]:
# Compute other adversarial examples
embeddings_train_adv = {}
for eps in train_adv:
    print("\nComputing train adversarial embeddings for eps=%.3f" % eps)
    embeddings_train_adv[eps] = Parallel(n_jobs=config.n_jobs)(
        delayed(embedding_getter)(line) for line in train_adv[eps])
print("\nComputing train clean embeddings")
embeddings_train_clean = Parallel(n_jobs=config.n_jobs)(
        delayed(embedding_getter)(line)
        for line in train_clean)
print("\nComputing test clean embeddings")
embeddings_test_clean = Parallel(n_jobs=config.n_jobs)(
        delayed(embedding_getter)(line)
        for line in test_clean)

In [ ]:
# The real deal: try to detect adversarial examples from normal examples
from tda.protocol import evaluate_embeddings

param_space = [{"M": 20, "sigma": sigma} for sigma in np.logspace(-3, 3, 7)]
kernel_type = KernelType.SlicedWasserstein
evaluation_results = evaluate_embeddings(embeddings_train_clean,
                                         embeddings_test_clean,
                                         embeddings_train_adv,
                                         embeddings_test_adv,
                                         kernel_type=kernel_type,
                                         param_space=param_space)

In [ ]:
# visualize embeddings
import matplotlib as mpl
import matplotlib.cm as cm

import seaborn as sns

cmap = cm.Blues_r

colors = {0: "b",
          0.1: "c",
          0.2: "m",
          0.3: "r"}
_, (ax1, ax3, ax4) = plt.subplots(1, 3, figsize=(15, 5))
for eps in embeddings:
    if not eps in [0., 0.1, 0.3]: continue
    color = colors[eps]
    for x in embeddings[eps]:
        birth, death = np.transpose(x)
        age = death - birth
        ax1.plot(birth, c=color)
        ax1.set_ylabel("birth")
        ax1.set_xlabel("points")
        # ax2.plot(death, c=color)
        # ax2.set_ylabel("death")
        # ax2.set_xlabel("points")
        ax3.plot(age, c=color)
        ax3.set_ylabel("age (death - birth)")
        ax3.set_xlabel("points")
        ax4.scatter(birth, death, c=color);
        ax4.set_xlabel("birth")
        ax4.set_ylabel("death")
plt.tight_layout()

In [ ]:
# Plot performance of the detector
import pandas as pd
import seaborn as sns

df = []
for key in ["supervised_metrics", "unsupervised_metrics"]:
    tmp = evaluation_results[key]
    if key == "unsupervised_metrics":
        sup = False
    else:
        sup = True
    for eps in tmp:
        df.append(dict(sup=sup, eps=eps, auc=tmp[eps]["auc"]["value"],
                       method="PersistentDiagram",
                       arch=architecture.name))
df = pd.DataFrame(df)

In [ ]:
sns.pointplot(data=df, x="eps", y="auc", hue="sup");

In [ ]:
df

In [ ]:
import tda.embeddings.persistence_landscape as pls
reload(pls)

persimgs = {}
for eps in sorted(list(embeddings_train_adv.keys())):
    persimgs[eps] = pls.persistence_diagrams_to_images(
        embeddings_train_adv[eps][::10], n_jobs=config.n_jobs,
        bandwidth=1e-4, backend="persim")

In [ ]:
from ipywidgets import interact

eps_list = np.array(list(persimgs.keys()))

@interact
def plot_persistence_images(eps=(0, .5, 0.05),
                            example_index=(0, len(persimgs[eps_list[0]]) - 1)):
    _, ax = plt.subplots(1, 1, figsize=(5, 5))
    closest_i = np.abs(eps - eps_list).argmin()
    eps = eps_list[closest_i]
    persimg = persimgs[eps][example_index]
    ax.imshow(persimg, cmap=plt.get_cmap("plasma"));
    ax.axis("off");
    ax.matshow(persimg)
    # plt.subplots_adjust(wspace=0, hspace=0)
    # plt.tight_layout()

In [ ]:
import pandas as pd
import tda.refactor.eval as _eval
reload(_eval)

scores = []
for supervised in [True, False][:1]:
    for kernel_type in [KernelType.PersistenceLandscape,
                        KernelType.SlicedWasserstein][1:]:
        if kernel_type == KernelType.PersistenceLandscape:
            param_space = [{"backend": "persim", "resolution": (k, k),
                            "bandwidth": bw}
                           for k in [5, 10, 20][2:] for bw in [1, 3]]
        elif kernel_type == KernelType.SlicedWasserstein:
            param_space = [dict(M=20, sigma=sigma)
                           for sigma in np.logspace(-3, 3, 7)]
        else:
            raise NotImplementedError(kernel_type)
        df = _eval.evaluate_embeddings(kernel_type,
                                       embeddings_train_clean,
                                       embeddings_test_clean,
                                       embeddings_train_adv,
                                       embeddings_test_adv,
                                       param_space,
                                       supervised=supervised,
                                       n_jobs=config.n_jobs,
                                       random_state=0)
        scores.append(df)
scores = pd.concat(scores)

In [ ]:
from ipywidgets import interact
import seaborn as sns

df = []
for _, line in scores.iterrows():
    line = line.to_dict()
    low = line.pop("auc.lower_bound")
    val = line.pop("auc.value")
    high = line.pop("auc.upper_bound")
    df += [{"quantile": q, "auc": val, **line}
            for q, val in zip(["lower", "middle", "upper"],
                              [low, val, high])]
df = pd.DataFrame(df)
    
@interact
def plot_scores(supervised=df.supervised.unique()):
    sns.catplot(data=df.loc[df.supervised == supervised], x="eps",
                y="auc", hue="kernel_type", kind="point");

In [ ]:
from scipy.sparse import bmat as sparse_bmat, coo_matrix


def toeplitz_1_ch(kernel, input_size):
    # shapes
    k_h, k_w = kernel.shape
    i_h, i_w = input_size
    o_h, o_w = i_h-k_h+1, i_w-k_w+1

    # construct 1d conv toeplitz matrices for each row of the kernel
    toeplitz = []
    for r in range(k_h):
        toeplitz.append(linalg.toeplitz(c=(kernel[r,0], *np.zeros(i_w-k_w)),
                                        r=(*kernel[r], *np.zeros(i_w-k_w))) ) 

    # construct toeplitz matrix of toeplitz matrices (just for padding=0)
    h_blocks, w_blocks = o_h, i_h
    h_block, w_block = toeplitz[0].shape

    W_conv = np.zeros((h_blocks, h_block, w_blocks, w_block))

    for i, B in enumerate(toeplitz):
        for j in range(o_h):
            W_conv[j, :, i+j, :] = B

    W_conv.shape = (h_blocks*h_block, w_blocks*w_block)

    return W_conv


def toeplitz_mult_ch(kernel, input_size, strides=1, padding=0):
    """Compute toeplitz matrix for 2d conv with multiple in and out channels.
    Args:
        kernel: shape=(n_out, n_in, H_k, W_k)
        input_size: (n_in, H_i, W_i)"""

    kernel_size = kernel.shape
    output_size = (kernel_size[0], input_size[1] - (kernel_size[1] - 1),
                   input_size[2] - (kernel_size[2] - 1))
    T = []
    for i, ks in enumerate(kernel):  # loop over output channel
        T_ = []
        for j, k in enumerate(ks):  # loop over input channel
            Tk = toeplitz_1_ch(k, input_size[1:])
            Tk = coo_matrix(Tk)
            T_.append(Tk)
        T.append(T_)
    return sparse_bmat(T)

In [ ]:
input_size = (6, 10, 10)
kernel_size = (7, input_size[0], 3, 3)
k = np.random.randn(np.prod(kernel_size)).reshape(kernel_size)
mat = toeplitz_mult_ch(k, input_size)
print(mat.shape)
# plt.matshow(mat); plt.axis("off");

In [ ]:
plt.matshow(mat.todense().T);
plt.axis("off");

In [ ]:
plt.matshow(architecture.layers[0].build_matrix().todense().T); plt.axis("off");

In [ ]:
conv = architecture.layers[0]

for param_ in conv.func.named_parameters():
    param = param_[1]
    # name_ = param_[0]
    if len(param.size()) > 1:
        kernel = param.data
        break
else:
    raise RuntimeError
kernel = kernel.cpu().numpy()

In [ ]:
input_size = (1, 28, 28)
mat = toeplitz_mult_ch(kernel, input_size)
plt.matshow(mat.todense().T);
plt.axis("off");


In [ ]:
np.max(mat.todense() - architecture.layers[0].build_matrix().todense())

In [ ]:
model(X_train).argmax(1)

In [ ]:
len(dataset.train_dataset)

In [ ]:
raw_thresholds.split("_")


In [ ]:
architecture.layers

In [ ]:
a, b = zip(*dataset.train_dataset)

In [ ]:
torch.stack(a).shape

In [ ]:
X_train.shape

In [ ]:
architecture

In [ ]:
architecture.forward(X_train[:1])

In [ ]:
len(embeddings_train_adv[eps][2])

In [ ]:
config.thresholds


In [8]:
config

Config(all_epsilons='0.01;0.1', architecture='mnist_lenet', attack_type='FGSM', dataset='MNIST', dataset_size=100, embedding_type='PersistentDiagram', epochs=50, kernel_type='SlicedWasserstein', n_jobs=24, noise=0.0, num_iter=50, sigmoidize=False, threshold_strategy='ActivationValue', thresholds='0.1_0.1_0.1_0.1')